In [105]:
import clr
from bonsai_pylib import load_bonsai_config
load_bonsai_config()

import System
# Add reference to the Harp package
clr.AddReference("Harp.Behavior")
import Harp.Behavior as behavior
clr.AddReference("Bonsai.Harp")
import Bonsai.Harp as harp


In [5]:
## Connect to the device usign the Async Device class
dev = behavior.AsyncDevice("COM5")
# Call the Dispose method to close the connection
#dev.Dispose()

In [17]:
# Get the map of available registers
RegisterMap = behavior.Device().RegisterMap
AvailableKeys = list(behavior.Device().RegisterMap.Keys)

# Get the Name + number of the register:
[print((RegisterMap[idx], idx)) for idx in AvailableKeys]

('WhoAmI', 0)
('HardwareVersionHigh', 1)
('HardwareVersionLow', 2)
('AssemblyVersion', 3)
('CoreVersionHigh', 4)
('CoreVersionLow', 5)
('FirmwareVersionHigh', 6)
('FirmwareVersionLow', 7)
('TimestampSeconds', 8)
('TimestampMicroseconds', 9)
('OperationControl', 10)
('ResetDevice', 11)
('DeviceName', 12)
('SerialNumber', 13)
('ClockConfiguration', 14)
('PortDigitalInput', 32)
('OutputSet', 34)
('OutputClear', 35)
('OutputToggle', 36)
('OutputState', 37)
('PortDIOSet', 38)
('PortDIOClear', 39)
('PortDIOToggle', 40)
('PortDIOState', 41)
('PortDIODirection', 42)
('PortDIOStateEvent', 43)
('AnalogData', 44)
('OutputPulseEnable', 45)
('PulseDOPort0', 46)
('PulseDOPort1', 47)
('PulseDOPort2', 48)
('PulseSupplyPort0', 49)
('PulseSupplyPort1', 50)
('PulseSupplyPort2', 51)
('PulseLed0', 52)
('PulseLed1', 53)
('PulseRgb0', 54)
('PulseRgb1', 55)
('PulseDO0', 56)
('PulseDO1', 57)
('PulseDO2', 58)
('PulseDO3', 59)
('PwmFrequencyDO0', 60)
('PwmFrequencyDO1', 61)
('PwmFrequencyDO2', 62)
('PwmFrequency

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [10]:
## A simple example on how to access all registers in loop
for key in AvailableKeys:
    method = f"Read{RegisterMap[key]}Async"
    try:
        result = getattr(dev, method)().Result
        print(result)
    except:
        pass

1216
0
444
29422
Bonsai.Harp.OperationControlPayload
BootFromDefault
Behavior
65535
RepeaterCapability, GeneratorCapability, ClockUnlock
0
0
0
0
0
0
0
0
0
0
DIO0, DIO1, DIO2
Harp.Behavior.AnalogDataPayload
SupplyPort0, SupplyPort1, SupplyPort2
500
500
500
15
15
15
500
500
500
500
250
250
250
250
1000
2000
3000
4000
50
50
50
50
0
0
Harp.Behavior.RgbAllPayload
Harp.Behavior.RgbPayload
Harp.Behavior.RgbPayload
10
10
30
30
PortDI, PortDIO, AnalogData, Camera0, Camera1
0
0
0
0
0
0
30
0
30
20000
1500
20000
1500
0
0
None
None
None
None
None
None
1


In [122]:
# Make an asynchronous call to read a register from the device
Cam0Freq = dev.ReadCamera0FrequencyAsync().Result
print([type(Cam0Freq), Cam0Freq])

# Some registers returned complex parsed values that can be accessed through properties
DIO = dev.ReadPortDigitalInputAsync().Result
print([type(DIO), DIO])
print(DIO) # Gets the value of the payload
print(type((DIO) & (DIO.DI0)), DIO.HasFlag(DIO.DI0)) #Extracts the value of the 0th bit using the mask
# You can also use:
#print(type((DIO) & (DIO.DI0)), int(((DIO) & (DIO.DI0))) == 1)



AggregateException: One or more errors occurred. ---> System.InvalidOperationException: The port is closed.
   at System.IO.Ports.SerialPort.Write(Byte[] buffer, Int32 offset, Int32 count)
   at Bonsai.Harp.AsyncDevice.<CommandAsync>d__54.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at System.Runtime.ExceptionServices.ExceptionDispatchInfo.Throw()
   at System.Runtime.CompilerServices.TaskAwaiter.HandleNonSuccessAndDebuggerNotification(Task task)
   at Harp.Behavior.AsyncDevice.<ReadCamera0FrequencyAsync>d__150.MoveNext()
   --- End of inner exception stack trace ---
   at System.Threading.Tasks.Task`1.GetResultCore(Boolean waitCompletionNotification)
---> (Inner Exception #0) System.InvalidOperationException: The port is closed.
   at System.IO.Ports.SerialPort.Write(Byte[] buffer, Int32 offset, Int32 count)
   at Bonsai.Harp.AsyncDevice.<CommandAsync>d__54.MoveNext()
--- End of stack trace from previous location where exceptio

In [50]:
# Return an harp message from an async read
# Note, the correct method still needs to be applied (i.e. "ReadByte")
harpMessage = dev.CommandAsync(harp.HarpCommand.ReadByte(behavior.PortDigitalInput.Address)).Result
print(harpMessage)
raw_payload = harpMessage.GetPayload()
print(bytearray(raw_payload.get_Array()))

Read 32 TimestampedU8@1821.44349 Length:1
bytearray(b'\x01\x0b \xff\x11\x1d\x07\x00\x00#6\x00\xb9')


In [64]:
# Return an harp message from an async read
# Note, here we use the parse-like method "GetPayload" to automatically get the value
harpMessage = dev.CommandAsync(harp.HarpCommand.ReadByte(behavior.PortDigitalInput.Address)).Result
print(harpMessage)
parsed_msg = (behavior.PortDigitalInput.GetPayload(harpMessage))
print([type(parsed_msg), parsed_msg])



Read 32 TimestampedU8@2087.50749 Length:1
[<class 'Harp.Behavior.DigitalInputs'>, <Harp.Behavior.DigitalInputs object at 0x00000220225E2D40>]
0
DI1
DI0


In [114]:
## Write to a register
Cam0Freq_0 = dev.ReadCamera0FrequencyAsync().Result

value = 30
Cam0Freq = dev.WriteCamera0FrequencyAsync(value).Result
# Confirm the value of the register has changed by reading it back
Cam0Freq = dev.ReadCamera0FrequencyAsync().Result
print(f"Initial value was: {Cam0Freq_0}, Write value: {value}, Read value: {Cam0Freq}")

Initial value was: 20, Write value: 30, Read value: 30


In [116]:
# Write an harp message directly
# Note, the correct method still needs to be applied (i.e. "WriteUInt16")
harpMessage = dev.CommandAsync(
    harp.HarpCommand.WriteUInt16(
        behavior.Camera0Frequency.Address, 100)).Result
print(harpMessage)

Write 93 TimestampedU16@3546.91648 Length:1


In [10]:
# Write an harp message directly without knowing the type by using the FromPayload method
msg = behavior.Camera0Frequency.FromPayload(harp.MessageType.Write, 100)

harpMessage = dev.CommandAsync(
    msg).Result
print(harpMessage)

Write 34 TimestampedU16@14.29248 Length:1


In [11]:
# Manipulate HarpMessage
an_harp_message = harp.HarpMessage.FromSingle(32, harp.MessageType.Write, 1.0)
print(harp.HarpMessage.get_Address(an_harp_message))
print(harp.HarpMessage.get_PayloadType(an_harp_message))
print(harp.HarpMessage.GetPayloadSingle(an_harp_message))

32
Float
1.0


In [121]:
dev.Dispose() # Make sure to dispose the device when done
